In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np

In [2]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

#Extraer URLs de anuncios de una página de listado
def obtener_urls_anuncios(url_listado):
    response = requests.get(url_listado, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    elementos = soup.find_all(['a', 'h2'], {'class': ['clp-listing-image-link', 'publication-title-list']})

    urls_anuncios = []
    for elemento in elementos:
        url = elemento.get('href')
        if url:
            url_completa = "https://chilepropiedades.cl" + url
            urls_anuncios.append(url_completa)

    time.sleep(1)  # Agrega un retardo de 4 segundos para evitar ser bloqueado
    
    return urls_anuncios

#Extraer información de un anuncio
def obtener_info_anuncio(url_anuncio):
    response = requests.get(url_anuncio, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Obtiene el valor de la UF
    valor_uf_element = soup.find('div', {'class': 'calculated-value-conversion'})
    if valor_uf_element is not None:
        valor_uf_text = valor_uf_element.get_text(strip=True)
        start_index = valor_uf_text.find("1 UF = $ ") + len("1 UF = $ ")
        first_two_chars = valor_uf_text[start_index: start_index + 2]
        last_three_chars = valor_uf_text[-3:]
        valor_uf_str = first_two_chars + last_three_chars
        try:
            valor_uf = int(valor_uf_str)
        except ValueError:
            valor_uf = np.nan
    else:
        valor_uf = np.nan

    value_tags = soup.find_all('div', {'class': 'clp-description-value'}) 
    label_tags = soup.find_all('div', {'class': 'clp-description-label'}) 

    values = [tag.text.strip() for tag in value_tags]
    labels = [tag.text.strip() for tag in label_tags]

    data = {
        "Valor": np.nan,
        "Valor(UF)": np.nan,
        "Gastos Comunes": np.nan,
        "Habitaciones": np.nan,
        "Baño": np.nan,
        "Estacionamientos": np.nan,
        "Amoblado": np.nan,
        "Superficie Total": np.nan,
        "Superficie Construida": np.nan,
        "Dirección": np.nan,
        "Código aviso": np.nan,
        "Código externo aviso": np.nan,
        "Tipo de publicación": np.nan,
        "Tipo de propiedad": np.nan,
        "Fecha Publicación": np.nan,
        "Valor UF": valor_uf,
        "Vigencia": "Activo",
        "URL": url_anuncio
    }

    for label, value in zip(labels, values):
        combined_string = label + " " + value
        for key in data.keys():
            if key == "Valor" and combined_string.startswith("Valor:"):
                data[key] = combined_string
            elif key == "Valor(UF)" and combined_string.startswith("Valor (UF aprox.)*:"):
                data[key] = combined_string
            elif key in combined_string and key != "Valor" and key != "Valor(UF)":
                data[key] = combined_string
    
    time.sleep(1)  # Agrega un retardo de 4 segundos
    return data

def obtener_numero_paginas():
    url = "https://chilepropiedades.cl/propiedades/arriendo-mensual/casa/region-metropolitana-de-santiago-rm/"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    respuesta = requests.get(url, headers=headers)
    bsoup = BeautifulSoup(respuesta.text, 'html.parser')

    texto = bsoup.find('div', class_='clp-results-text-container d-none d-sm-block col-sm-6 text-right').span.text
    num_paginas = texto.split(":")[1].strip()

    return int(num_paginas)

# Ahora juntamos todo en una función de alto nivel
def webscraping_propiedades():
    num_paginas = obtener_numero_paginas()
    url_base = "https://chilepropiedades.cl/propiedades/arriendo-mensual/casa/region-metropolitana-de-santiago-rm/{}"
    data = []
    last_percentage = 0

    # Parte 1: Iterar sobre las páginas de listado
    for i in range(num_paginas):
        porcentaje_completado = round((i + 1) / num_paginas * 100, -1)  # Redondea al 10% más cercano
        # Solo imprime si el porcentaje completado es 10% mayor que el último porcentaje impreso
        if porcentaje_completado >= last_percentage + 10:
            print(f"{porcentaje_completado}% Completado")
            last_percentage = porcentaje_completado  # Actualiza el último porcentaje impreso

        url_listado = url_base.format(i)

        # Parte 2: Obtener URLs de los anuncios en la página
        urls_anuncios = obtener_urls_anuncios(url_listado)

        # Parte 3: Obtener la información de cada anuncio
        for url_anuncio in urls_anuncios:
            info_anuncio = obtener_info_anuncio(url_anuncio)
            data.append(info_anuncio)

    df = pd.DataFrame(data)

    # Limpieza de datos
    for col in df.columns:
        df[col] = df[col].replace(col + ': ', '', regex=True)
    # Eliminando columna "Valor(UF)" y "Código externo aviso"
    df = df.drop('Valor(UF)', axis=1)
    df = df.drop('Código externo aviso', axis=1)
    # Reemplazando caracteres
    df['Valor'] = df['Valor'].str.replace('.', '', regex=False)
    df['Valor'] = df['Valor'].str.replace(',', '.', regex=False)
    df['Valor'] = df['Valor'].str.replace('$ ', '', regex=False)
    df['Gastos Comunes'] = df['Gastos Comunes'].astype(str)
    df['Gastos Comunes'] = df['Gastos Comunes'].str.replace('.', '', regex=False)
    df['Gastos Comunes'] = df['Gastos Comunes'].str.replace('$ ', '', regex=False)
    df['Superficie Total'] = df['Superficie Total'].str.replace(' m²', '', regex=False)
    df['Superficie Construida'] = df['Superficie Construida'].str.replace(' m²', '', regex=False)

    # Elimino columna "Gastos Comunes"
    df = df.drop('Gastos Comunes', axis=1)
    
    # Leemos el archivo histórico df2
    df2 = pd.read_excel('casas_data.xlsx')

    # Si df2 no tiene una columna de "Vigencia", la agregamos
    if 'Vigencia' not in df2.columns:
        df2['Vigencia'] = "Inactivo"

    # Actualizamos la vigencia de las filas que existen en ambos df y df2
    df2.loc[df2['URL'].isin(df['URL']), 'Vigencia'] = "Activo"

    # Agregamos las nuevas filas a df2
    df2 = pd.concat([df2, df[~df['URL'].isin(df2['URL'])]], ignore_index=True)

    # Guardamos el DataFrame df2 actualizado
    df2.to_excel('casas_data.xlsx', index=False)

# Uso de la función
webscraping_propiedades()

10.0% Completado
20.0% Completado
30.0% Completado
40.0% Completado
50.0% Completado
60.0% Completado
70.0% Completado
80.0% Completado
90.0% Completado
100.0% Completado


In [3]:
df5 = pd.read_excel('casas_data.xlsx')

#convertimos la columna valor a tipo string
df5['Valor'] = df5['Valor'].astype(str)
#rellena los valores faltantes de la columna con el valor de la fila anterior
df5['Valor UF'] = df5['Valor UF'].fillna(method='ffill')

#si en columna valor hay un valor que empieza con la cadena "UF " entonces se elimina esa cadena y se multiplica por el valor de la UF
def valor_uf(valor):
    if valor.startswith('UF '):
        valor = valor.replace('UF ', '')
        valor = float(valor)
        valor = valor * df5['Valor UF'][0]
    elif valor.startswith('USD '):
        valor = valor.replace('USD ', '')
    else:
        return valor
    return valor
#aplicamos la función a la columna valor
df5['Valor'] = df5['Valor'].apply(valor_uf)
#convertimos la columna valor a tipo string
df5['Valor'] = df5['Valor'].astype(str)
#eliminamos espacios en blanco de la columna valor
df5['Valor'] = df5['Valor'].str.replace(' ', '', regex=False)
#convertimos a tipo float
df5['Valor'] = df5['Valor'].astype(float)
df5.to_excel('casas_data.xlsx', index=False)